In [1]:
import os
import cv2
from ultralytics import YOLO
import pandas as pd
from sklearn.model_selection import train_test_split


In [2]:
train_folder = "dataset/train"
test_folder = "dataset/test"
valid_folder = "dataset/valid"

train_annotation = os.path.join(train_folder, "_annotations.csv")
test_annotation = os.path.join(test_folder, "_annotations.csv")
valid_annotation = os.path.join(valid_folder, "_annotations.csv")


In [3]:
def load_annotations(csv_file):
    return pd.read_csv(csv_file)

train_annotations = load_annotations(train_annotation)
test_annotations = load_annotations(test_annotation)
valid_annotations = load_annotations(valid_annotation)

print("Train Annotations:")
print(train_annotations.head())


Train Annotations:
                                           filename  width  height  \
0  101_jpeg.rf.0a1b2b7c9c5dc78d68e338c00628d072.jpg    640     640   
1  149_jpeg.rf.12791a08af9b198a4975c55cc72bd61c.jpg    640     640   
2  140_jpeg.rf.02c767f81d5d7d18f9c211d633a58f6f.jpg    640     640   
3  140_jpeg.rf.02c767f81d5d7d18f9c211d633a58f6f.jpg    640     640   
4  140_jpeg.rf.02c767f81d5d7d18f9c211d633a58f6f.jpg    640     640   

             class  xmin  ymin  xmax  ymax  
0  old-aged-person    22     3   400   640  
1  old-aged-person    26    70   309   489  
2  old-aged-person   464     3   622   303  
3  old-aged-person   335    34   442   297  
4  old-aged-person   221     5   338   224  


In [4]:
import yaml

base_path = 'dataset'

train_path = os.path.join(base_path, 'train', 'image')
valid_path = os.path.join(base_path, 'valid', 'image')
test_path = os.path.join(base_path, 'test', 'image')

classes = ['old-aged-person', 'cane', 'wheelchair']

data_dict = {
    'train': train_path,
    'val': valid_path,
    'test': test_path,
    'nc': len(classes),
    'names': classes,
}

yaml_file_path = os.path.join(base_path, 'datasett.yaml')
with open(yaml_file_path, 'w') as file:
    yaml.dump(data_dict, file)

print(f'datasett.yaml created at: {yaml_file_path}')


datasett.yaml created at: dataset/datasett.yaml


In [5]:
csv_file = 'dataset/valid/_annotations.csv'
image_folder = 'dataset/valid/image'
output_label_folder = 'dataset/valid/image'

os.makedirs(output_label_folder, exist_ok=True)

class_mapping = {
    'old-aged-person': 0,
    'cane': 1,
    'wheelchair': 2
}

df = pd.read_csv(csv_file)

In [6]:
for index, row in df.iterrows():
    filename = row['filename']
    width = row['width']
    height = row['height']
    class_name = row['class']
    xmin = row['xmin']
    ymin = row['ymin']
    xmax = row['xmax']
    ymax = row['ymax']

    x_center = (xmin + xmax) / 2 / width
    y_center = (ymin + ymax) / 2 / height
    bbox_width = (xmax - xmin) / width
    bbox_height = (ymax - ymin) / height

    class_id = class_mapping.get(class_name, 'unknown')
    
    if class_id == 'unknown':
        continue

    yolo_annotation = f"{class_id} {x_center} {y_center} {bbox_width} {bbox_height}\n"
    
    label_filename = os.path.join(output_label_folder, os.path.splitext(filename)[0] + '.txt')
    
    with open(label_filename, 'a') as f:
        f.write(yolo_annotation)

print("Conversion complete.")


Conversion complete.


In [ ]:
import albumentations as A
output_augmented_folder = 'dataset/valid/augmented_image'

os.makedirs(output_augmented_folder, exist_ok=True)

transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.Blur(blur_limit=3, p=0.2),
    A.ColorJitter(p=0.2),
    A.RandomBrightnessContrast(p=0.2),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=15, p=0.5)
])

image_path = os.path.join(image_folder, filename)
image = cv2.imread(image_path)

augmented = transform(image=image)
augmented_image = augmented['image']

augmented_image_filename = f"aug_{filename}"
augmented_image_path = os.path.join(output_augmented_folder, augmented_image_filename)
cv2.imwrite(augmented_image_path, augmented_image)

In [10]:
model = YOLO("yolov8n.pt")

model.train(data='dataset/datasett_augmented.yaml', epochs=50)

model.save('yolov8.pt')



New https://pypi.org/project/ultralytics/8.3.11 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.2 🚀 Python-3.12.2 torch-2.4.1 CPU (Apple M2)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=dataset/datasett_augmented.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train59, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_tx

train: Scanning /Users/hamooshaq/projek/sistem_disabilitas1/dataset/train/image.cache... 113 images, 1 backgrounds, 0 corrupt: 100%|██████████| 114/114 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



val: Scanning /Users/hamooshaq/projek/sistem_disabilitas1/dataset/valid/image.cache... 33 images, 0 backgrounds, 0 corrupt: 100%|██████████| 33/33 [00:00<?, ?it/s]

val: WARNING ⚠️ /Users/hamooshaq/projek/sistem_disabilitas1/dataset/valid/image/103_jpeg.rf.a8c06e583716f458c1ae5bc7e0374fdd.jpg: 13 duplicate labels removed
val: WARNING ⚠️ /Users/hamooshaq/projek/sistem_disabilitas1/dataset/valid/image/107_jpeg.rf.2293c0d43492848eb9f626c605b6d246.jpg: 26 duplicate labels removed
val: WARNING ⚠️ /Users/hamooshaq/projek/sistem_disabilitas1/dataset/valid/image/109_jpeg.rf.c184b440f35cef017cbeaac2f6f8afec.jpg: 26 duplicate labels removed
val: WARNING ⚠️ /Users/hamooshaq/projek/sistem_disabilitas1/dataset/valid/image/110_jpeg.rf.04893295ed99d1911778b9310ff5be59.jpg: 26 duplicate labels removed
val: WARNING ⚠️ /Users/hamooshaq/projek/sistem_disabilitas1/dataset/valid/image/114_jpeg.rf.0aae0f80345417732f8f5e6b08d091db.jpg: 39 duplicate labels removed
val: WARNING ⚠️ /Users/hamooshaq/projek/sistem_disabilitas1/dataset/valid/image/119_jpeg.rf.c29ecd9f6cc750a3e51de6a27251ca40.jpg: 26 duplicate labels removed
val: WARNING ⚠️ /Users/hamooshaq/projek/sistem_disab

...

      50/50         0G     0.9739      1.367      1.274          5        640: 100%|██████████| 8/8 [01:08<00:00,  8.57s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:06<00:00,  3.35s/it]

                   all         33         58      0.798      0.647      0.729      0.334



50 epochs completed in 1.878 hours.
Optimizer stripped from runs/detect/train59/weights/last.pt, 5.6MB
Optimizer stripped from runs/detect/train59/weights/best.pt, 5.6MB

Validating runs/detect/train59/weights/best.pt...
Ultralytics 8.3.2 🚀 Python-3.12.2 torch-2.4.1 CPU (Apple M2)
Model summary (fused): 186 layers, 2,684,953 parameters, 0 gradients, 6.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:06<00:00,  3.12s/it]


                   all         33         58      0.545       0.67      0.661      0.363
       old-aged-person         18         31      0.598      0.452      0.493      0.164
                  cane         15         20      0.479        0.7      0.655      0.354
            wheelchair          7          7      0.559      0.857      0.836      0.571
Speed: 1.1ms preprocess, 181.0ms inference, 0.0ms loss, 3.5ms postprocess per image
Results saved to runs/detect/train59


In [11]:
results = model.val()


Ultralytics 8.3.2 🚀 Python-3.12.2 torch-2.4.1 CPU (Apple M2)
Model summary (fused): 186 layers, 2,684,953 parameters, 0 gradients, 6.8 GFLOPs


val: Scanning /Users/hamooshaq/projek/sistem_disabilitas1/dataset/valid/image.cache... 33 images, 0 backgrounds, 0 corrupt: 100%|██████████| 33/33 [00:00<?, ?it/s]

val: WARNING ⚠️ /Users/hamooshaq/projek/sistem_disabilitas1/dataset/valid/image/103_jpeg.rf.a8c06e583716f458c1ae5bc7e0374fdd.jpg: 13 duplicate labels removed
val: WARNING ⚠️ /Users/hamooshaq/projek/sistem_disabilitas1/dataset/valid/image/107_jpeg.rf.2293c0d43492848eb9f626c605b6d246.jpg: 26 duplicate labels removed
val: WARNING ⚠️ /Users/hamooshaq/projek/sistem_disabilitas1/dataset/valid/image/109_jpeg.rf.c184b440f35cef017cbeaac2f6f8afec.jpg: 26 duplicate labels removed
val: WARNING ⚠️ /Users/hamooshaq/projek/sistem_disabilitas1/dataset/valid/image/110_jpeg.rf.04893295ed99d1911778b9310ff5be59.jpg: 26 duplicate labels removed
val: WARNING ⚠️ /Users/hamooshaq/projek/sistem_disabilitas1/dataset/valid/image/114_jpeg.rf.0aae0f80345417732f8f5e6b08d091db.jpg: 39 duplicate labels removed
val: WARNING ⚠️ /Users/hamooshaq/projek/sistem_disabilitas1/dataset/valid/image/119_jpeg.rf.c29ecd9f6cc750a3e51de6a27251ca40.jpg: 26 duplicate labels removed
val: WARNING ⚠️ /Users/hamooshaq/projek/sistem_disab


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:11<00:00,  4.00s/it]


                   all         33         58      0.545       0.67      0.661      0.363
       old-aged-person         18         31      0.598      0.452      0.493      0.164
                  cane         15         20      0.479        0.7      0.655      0.354
            wheelchair          7          7      0.559      0.857      0.836      0.571
Speed: 3.1ms preprocess, 345.6ms inference, 0.0ms loss, 5.9ms postprocess per image
Results saved to runs/detect/train592


In [ ]:
import cv2
from ultralytics import YOLO


model = YOLO('yolov8n.pt')  

cap = cv2.VideoCapture(0)

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

output_filename = 'detected_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'XVID') 
out = cv2.VideoWriter(output_filename, fourcc, 20.0, (frame_width, frame_height))

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    results = model(frame)

    annotated_frame = results[0].plot()

    out.write(annotated_frame)

    cv2.imshow("Webcam", annotated_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()
print("Video telah disimpan sebagai", output_filename)



0: 384x640 (no detections), 55.8ms
Speed: 2.3ms preprocess, 55.8ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)



OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


0: 384x640 (no detections), 68.3ms
Speed: 1.8ms preprocess, 68.3ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 60.1ms
Speed: 1.8ms preprocess, 60.1ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 55.9ms
Speed: 2.2ms preprocess, 55.9ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 57.8ms
Speed: 2.1ms preprocess, 57.8ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 57.7ms
Speed: 2.7ms preprocess, 57.7ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 78.8ms
Speed: 2.9ms preprocess, 78.8ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 81.6ms
Speed: 3.4ms preprocess, 81.6ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 100.2ms
Speed: 3.9ms preprocess, 100.2ms 

In [12]:
import cv2
from ultralytics import YOLO

model = YOLO('yolov8.pt')

cap = cv2.VideoCapture(0)

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

output_filename = 'video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'XVID') 
out = cv2.VideoWriter(output_filename, fourcc, 20.0, (frame_width, frame_height))

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    results = model(frame)

    annotated_frame = results[0].plot()

    out.write(annotated_frame)

    cv2.imshow("Webcam", annotated_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()
print("Video telah disimpan sebagai", output_filename)


2024-10-15 01:05:35.007 python[75701:1450240] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


Video telah disimpan sebagai video.mp4
